# Robust Knapsack

This tutorial shows how to construct and solve the Robust Portfolio problem, introduced by Bertsimas and Sim 2004 [1] (https://pubsonline.informs.org/doi/abs/10.1287/opre.1030.0065)
We show to how solve the robust knapsack problem (Section 6.1). Using the same formulation as in [1], the problem can be formulated as follows:

$$
\begin{array}{ll}
\text{maximize} & \mathbf{c}^T \mathbf{x} \\
\text{subject to} & \mathbf{w}^T\mathbf{x} \leq b \quad \forall w \in \mathcal{U}\\
                  & \mathbf{x}\in\left\{ 0,1\right\}^n
\end{array}
$$

TODO: change intro

$$
\mathcal{U} = \{w \mid \bar{w} - \delta \le w \le \bar{w} + \delta\}
$$

where there are $n$ items, $\mathbf{x}$ are the binary decision variables, their values are denoted by $\mathbf{c}$, and their weights $\mathbf{w}$ belong to a box uncertainty set, where the expected weights are denoted by $\mathbf{w_e}$, and their uncertainties are captured by $\pmb{\delta}$.

In [12]:
import cvxpy as cp
import numpy as np
import lropt

np.random.seed(seed=1234)

We define the constants as shown below:

In [13]:
n = 200 #Number of items
b = 1000 #Capacity
c = np.random.uniform(low=0., high=1., size=n) #Value of each item
w_e = np.random.uniform(low=1., high=2, size=n) #Mean weight of each item
delta = np.random.uniform(low=0., high=0.1, size=n) #Weights uncertainties
NUM_DEC = 3

The uncertain parameter $\mathbf{p}$ is formulated using LROPT in the block below. We use the box uncertainty set, which is defined as follows:

Box uncertainty: $\{ u = Az+b \mid\| z\|_\infty \leq \rho  \}$ 

- $A$ : np.array, optional
  * Scaling matrix for u. Default identity matrix.
- $b$ : np.array, optional
  * Relocation vector for u. Default None.

We define an uncertain parameter $\mathbf{p}$ using the Box definition with $\rho=1,\mathbf{A}=\text{diag}\left( \pmb{\delta}\right), \mathbf{b}=\mathbf{w_e}$.

In [14]:
#Define a helper uncertainty variable u
uncertainty_set = lropt.Box(rho=1, a=np.diag(delta), b=w_e)
w = lropt.UncertainParameter(n, uncertainty_set=uncertainty_set)
x = cp.Variable(n, boolean=True) #Optimization variable

#Define and solve the problem
objective = cp.Maximize(c@x)
constraints = [
                w@x <= b,
              ]
prob = lropt.RobustProblem(objective=objective, constraints=constraints)
print(round(prob.solve(solver = 'MOSEK'), NUM_DEC))

SolverError: Problem is mixed-integer, but candidate QP/Conic solvers ([]) are not MIP-capable.